# 성능개선 2 : XGBoost 모델

In [ ]:
import pandas as pd

In [ ]:
# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 피처 엔지니어링

### 데이터 합치기

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1)  # 타깃값 제거

all_features = all_data.columns  # 전체 피처
all_features

### 명목형 피처 원-핫 인코딩

모든 명목형 피처에 원-핫 인코딩을 적용

명목형 데이터에는 고윳값별 순서가 따로 없음

'cat' 이 포함된 피처가 명목형 피처임

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()  # 원-핫 인코더 객체 생성

# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

### 파생 피처 추가

1. 한 데이터가 가진 결측값 개수를 파생 피처로 만들자! -1 이 결측값이었으니 결측값 개수를 구하려면 -1 개수를 구하면 됨

In [ ]:
# '데이터 하나당 결측값 개수'를 파생 피처로 추가
all_data['num_missing'] = (all_data == -1).sum(axis=1)

In [ ]:
# 명목형 피처, calc 분류의 피처를 제외한 피처
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature)]

# num_missing 을 remaining_features 에 추가
remaining_features.append('num_missing')

2. ind 분류의 피처들을 살펴보자. 모든 ind 피처 값을 연결해서 새로운 피처를 만들려고 함
    - 예를 들어, ps_ind_01, ps_ind_02_cat, ps_ind_03 의 값이 각각 2, 2, 5라면 모든 값을 연결해 2_2_5_ 로 만듦
    - ind 피처가 총 18개이므로 18개 값이 연결된 새로운 피처를 만들고, 이 피처명을 'mix_ind' 라고 함

> 왜 이런 파생 피처들을 만들까?
> 이 파생 피처들은 타깃값 예측에 어떤 도움이 되는걸까? 사실 처음부터 파생 피처가 타깃값 예측에 도움이 되는지 알기는 쉽지 않음. 사칙연산도 해보고, 통계도 내보고, 문자열 연결도 해보는 등 갖은 방법으로 피처 엔지니어링을 해볼 수 있음. 실제로 많은 상위권 캐글러도 여러 피처 엔지니어링을 시도함. 그중 성능 향상에 도움되는 피처를 선별함. 앞의 결측값 개수나 뒤에서 만들 '명목형 피처의 고윳값별 개수' 피처도 어떤 이유에서 타깃값 예측에 도움을 주는지 단번에 알기는 어려움. 이런 방법도 있음을 기억해두고 다른 문제를 풀 때 응용해보자

In [ ]:
# 분류가 ind 인 피처
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [ ]:
all_data['mix_ind']

> 새로 만든 mix_ind 가 들쑥날쑥하게 한 칸씩 들여 써진 것처럼 보일 겁니다. 실제로 들여 써진건 아니고 all_data['mix_ind'] 를 호출하면 값을 오른쪽 정렬하기 때문에 그렇게 보이는 것임

3. 명목형 피처의 고윳값별 개수를 새로운 피처로 추가
    - 고윳값별 개수는 value_counts() 로 구함

In [ ]:
all_data['ps_ind_02_cat'].value_counts()

In [ ]:
all_data['ps_ind_02_cat'].value_counts().to_dict()

- 명목형 피처의 고윳값별 개수를 파생 피처로 만들자. cat 분류에 속하는 피처들(cat_features)과 mix_ind 피처를 모두 명목형 피처로 간주

In [ ]:
cat_count_features = []
for feature in cat_features + ['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])
    cat_count_features.append(f'{feature}_count')

In [ ]:
cat_count_features

### 필요 없는 피처 제거

지금까지 만든 피처는 다음과 같음

- encoded_cat_matrix: 원-핫 인코딩된 명목형 피처
- remaining_features: 명목형 피처와 calc 분류의 피처를 제외한 피처들 (+ num_missing)
- cat_count_features: mix_ind 를 포함한 명목형 피처의 고윳값별 개수 파생 피처

제거해야 할 피처

- 이진 피처 : ps_ind_10_bin ~ ps_ind_13_bin, ps_calc_15_bin ~ ps_calc_20_bin
- 순서형 피처 : ps_ind_14, ps_calc_04 ~ ps_calc_14
- 연속형 피처 : ps_calc_01 ~ ps_calc_03, ps_car_14

In [ ]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# remaining_features, cat_count_features 에서 drop_features 를 제거한 데이터
all_data_remaining = all_data[remaining_features + cat_count_features].drop(drop_features, axis=1)

In [ ]:
from scipy import sparse

In [ ]:
# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

지금까지 한 작업

1. 명목형 피처에 원-핫 인코딩을 적용
2. 데이터 하나당 가지고 있는 결측값 개수를 새로운 피처로 추가
3. 모든 ind 피처 값을 연결해서 새로운 명목형 피처를 만듦(직접 사용하진 않고, 4에서 활용하기 위해 만듦)
4. 명목형 피처의 고윳값별 개수를 새로운 피처로 추가
5. 필요 없는 피처를 제거(drop_features 와 calc 분류의 피처들)

### 데이터 나누기

In [ ]:
num_train = len(train)  # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

## 평가지표 계산 함수 작성

### 지니계수란?

원래 경제학에서 쓰는 용어로 소득 불평등 정도를 나타내는 지표임

지니계수가 작을수록 소득 수준이 평등하고, 클수록 불평등함을 의미

지니계수는 로렌츠 곡선을 이용해 계산하고 로렌츠 곡선을 그리려면 모든 경제인구를 소득 순서대로 나열한 후에 가로축은 인구 누적 비율, 세로축은 소득 누적 점유율로 설정

인구 누적 비율과 해당 소득 누적 점유율을 연결한 선을 로렌츠 곡선이라고 함

<img src=https://t1.daumcdn.net/cfile/blog/224431355845363417>

지니계수는 A 영역 넓이를 삼각형 전체 넓이로 나눈 값을 의미

A 영역이 좁을수록(로렌츠 곡선이 대각선과 가까워질수록) 소득 수준은 평등

반대로 A 영역이 넓을수록(로렌츠 곡선이 대각선과 멀어질수록) 소득 수준은 불평등

머신러닝에서 지니계수는 모델의 예측 성능을 측정하는데 쓰임

예측값을 크기순으로 정렬해서 로렌츠 곡선을 구함

> 지니계수 값은 (2 x ROC AUC - 1)과 같음. 그렇기 때문에 평가지표가 지니계수이면 평가지표가 ROC AUC 인 상황과 거의 비슷하긴 함

### 정규화 지니계수 계산 함수

정규화란 값의 범위를 0~1 사이로 조정한다는 뜻이므로, 정규화 지니계수는 값이 0에 가까울수록 성능이 나쁘고, 1에 가까울수록 성능이 좋다는 의미

$$정규화 지니계수 = \frac{예측 값에 대한 지니계수}{예측이 완벽할 떄의 지니계수}$$

'예측 값에 대한 지니계수' 는 예측값과 실제값으로 구한 지니계수

'예측이 완벽할 때의 지니계수' 는 실젯값과 실젯값으로 구한 지니계수를 뜻함

In [ ]:
import numpy as np

In [ ]:
def eval_gini(y_true, y_pred):
    # 실젯값과 예측값의 크기가 서로 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0]  # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples)  # 대각선 값
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]  # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)  # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)  # 예측값에 대한 지니계수
    
    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()]  # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order)  # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)  # 예측이 완벽할 때 지니계수
    
    # 정규화된 지니계수
    return G_pred / G_true

> 로직을 간단히만 설명해보자면 로렌츠 곡선 상단 넓이를 삼각형 넓이로 나눈 값이 지니계수임. 정규화 지니계수는 '예측값에 대한 지니계수'를 '예측이 완벽할 때의 지니계수'로 나눈 값임. 결국 로렌츠 곡선 상단 넓이를 구할 수 있으면 정규화 지니계수도 구할 수 있음

In [ ]:
# XGBoost 용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)  # 반환값(평가지표 이름, 평가 점수)

## 하이퍼파라미터 최적화

베이지안 최적화 기법을 활용해 하이퍼파라미터를 조정

그리드서치보다 더 빠르고 효율적이며, 코드도 직관적이어서 사용하기도 편리함

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = xgb.DMatrix(X_train, y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

### 하이퍼파라미터 범위 설정

1. 하이퍼파라미터 범위를 점점 좁히는 방법
    - 예컨대 0 ~ 1 범위의 하이퍼파라미터가 있다면 처음에는 범위를 0 ~ 1 전체로 잡고 베이지안 최적화를 수행
    - 0.5 를 최적하이퍼파라미터로 찾았다면 다시 0.5 주변으로 범위를 잡음
    - 가령 0.4 ~ 0.6 정도로 좁히는 방식을 반복하여 하이퍼파라미터 범위를 찾아줄 수 있음
    
2. 다른 상위권 캐글러가 설정한 하이퍼파라미터를 참고하는 방법
    - 공유된 코드를 참고해서 하이퍼파라미터 범위를 설정
    - 여러 차례 연습하다 보면 하이퍼파라미터에 대한 대략적인 감이 잡힘

In [ ]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'max_depth': (4, 8),
                'subsample': (0.6, 0.9),
                'colsample_bytree': (0.7, 1.0),
                'min_child_weight': (5, 7),
                'gamma': (8, 11),
                'reg_alpha': (7, 9),
                'reg_lambda': (1.1, 1.5),
                'scale_pos_weight': (1.4, 1.6),
               }

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary:logistic',
                'learning_rate': 0.02,
                'random_state': 1991,
               }

### (베이지안 최적화용) 평가지표 계산 함수 작성

LightGBM 용 eval_function() 과 유사하지만 다른점

1. 하이퍼파라미터명
2. train() 메서드 내 검증 데이터 전달 방식
3. train() 메서드 내 maximize 파라미터
4. predict() 메서드에 DMatrix 타입을 전달하는 점

In [ ]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                  reg_alpha, gamma, reg_lambda, scale_pos_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터
    params = {'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'min_child_weight': min_child_weight,
              'gamma': gamma,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': scale_pos_weight,
             }
    
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)
    
    # LightGBM 모델 훈련
    xgb_model = xgb.train(params=params,
                          dtrain=bayes_dtrain,
                          num_boost_round=2000,
                          evals=[(bayes_dvalid, 'bayes_dvalid')],
                          maximize=True,
                          feval=gini,
                          early_stopping_rounds=200,
                          verbose_eval=False,
                         )
    
    best_iter = xgb_model.best_iteration  # 최적 반복 횟수
    
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid,
                              iteration_range=(0, best_iter))

    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')
    
    return gini_score

### 최적화 수행

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,  # 평가지표 계산 함수
                                 pbounds=param_bounds,  # 하이퍼파라미터 범위
                                 random_state=0)

In [ ]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

### 결과 확인

In [ ]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

num_leaves 와 min_child_samples 는 원래 정수형 하이퍼파라미터이므로 정수형으로 변환하여 다시 저장

In [ ]:
# 정수형 하이퍼파라미터 변환
max_params['max_depth'] = int(round(max_params['max_depth']))

In [ ]:
# 여기에 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [ ]:
max_params

## 모델 훈련 및 성능 검증

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#' * 40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#' * 40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx]  # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx]  # 검증용 데이터
    
    # LightGBM 전용 데이터셋 생성
    dtrain = xgb.DMatrix(X_train, y_train)  # LightGBM 전용 훈련 데이터셋
    dvalid = xgb.DMatrix(X_valid, y_valid)  # LightGBM 전용 검증 
    dtest = xgb.DMatrix(X_test)
    
    # LightGBM 전용 데이터셋 생성
    xgb_model = xgb.train(params=max_params,  # 최적 하이퍼파라미터
                          dtrain=dtrain,  # 훈련 데이터셋
                          num_boost_round=2000,  # 부스팅 반복 횟수
                          evals=[(dvalid, 'valid')],  # 성능 평가용 검증 데이터셋
                          maximize=True,
                          feval=gini,  # 검증용 평가지표
                          early_stopping_rounds=200,  # 조기종료 조건
                          verbose_eval=100,  # 100번째마다 점수 출력
                         )
    
    # 모델 성능이 가장 좋을 떄의 부스팅 반복 횟수 저장
    best_iter = xgb_model.best_iteration
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += xgb_model.predict(dtest,
                                        iteration_range=(0, best_iter)) / folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid,
                                                  iteration_range=(0, best_iter))
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

In [ ]:
print('OOF 검증 데이터 지니계수 :', eval_gini(y, oof_val_preds))

## 예측 및 결과 제출

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')